In [1]:
import findspark

In [2]:
findspark.init('/home/huy/spark-2.2.3-bin-hadoop2.7')

Préparer les données pour Spark ML

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

# définir un SparkContext en local. Permet de travailler avec les RDD
# sc = SparkContext.getOrCreate()

# Construire une session Spark
# SparkSession est une couche supérieure à SparkContext
spark = SparkSession.builder.appName('Spark Context').getOrCreate()

spark

La base de données contient des caractéristiques audio des chansons.
Elle a 91 variables.

L'objectif est d'estimer l'année de sortie avec une régression linéaire.

In [4]:
df_raw = spark.read.csv("YearPredictionMSD.txt")

df_raw.sample(False, .00001, seed = 222).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c81,_c82,_c83,_c84,_c85,_c86,_c87,_c88,_c89,_c90
0,2007,49.83538,2.61013,17.00197,-9.50786,-23.94269,-14.72739,4.47792,4.06217,4.78611,...,10.59593,81.25012,33.10853,-7.46066,7.60570,91.36251,-66.34214,9.62220,119.81564,2.81304
1,2009,45.78269,34.74613,-8.52873,11.44336,-18.00745,-29.79324,11.83189,-5.26578,0.34463,...,8.23981,-120.67690,-56.06567,17.67264,8.23461,-46.53563,39.40150,9.44279,-24.93795,-8.67221
2,2007,43.61806,-12.13404,-21.41169,0.17262,23.94724,3.72902,13.61887,-3.44822,-2.09872,...,20.43382,-74.56965,59.65242,131.46903,6.07510,34.06221,-162.45943,2.73542,66.37659,-25.99604
3,2006,46.79964,54.54515,11.05619,2.00775,11.86970,-10.15977,6.25193,5.34281,1.45345,...,7.26790,-203.19105,84.51010,-73.76258,35.57993,59.14017,3.72889,-8.86871,45.47880,-11.22200
4,2006,52.00132,35.66485,34.37876,1.69847,-34.09650,-19.76410,13.33325,5.54795,6.81785,...,-17.10870,-121.60406,-31.96296,-40.16986,-5.51689,15.41615,-78.71597,2.63836,-8.23705,0.84104


## Mettre le jeu de données en format svmlib => dataframe avec deux colonnes label et features.

### Transformer toutes les colonnes à utiliser sous format numérique

Le parsing de PySpark enregistre tous les variables en string.

Il est possible d'automatiser le cast des colonnes avec la fonction col de pyspark.sql.functions

Le principe est de nommer une colonne et de répéter l'opération avec une boucle.

In [5]:
from pyspark.sql.functions import col

# pour ttes les colonnes de 1 à 91, cast en "double"
exprs = [col(c).cast("double") for c in df_raw.columns[1:91]]

# la première colonne est l'année et est en int
df = df_raw.select(df_raw._c0.cast("int"), *exprs)

df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tr

vérfier s'il y a des valeurs manquantes

In [6]:
df.describe().toPandas()

,summary,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,...,_c81,_c82,_c83,_c84,_c85,_c86,_c87,_c88,_c89,_c90
0,count,515345,515345,515345,515345,515345,515345,515345,515345,515345,...,515345,515345,515345,515345,515345,515345,515345,515345,515345,515345
1,mean,1998.3970815667174,43.3871256257654,1.2895541971106774,8.658347088513471,1.164124465959709,-6.553600704557125,-9.521975199837009,-2.391089424909519,-1.7932355097264998,...,15.755406044028707,-73.46149977267599,41.54242155146567,37.9341187352939,0.31575127167237227,17.669213222656687,-26.31533596198668,4.4586411071806475,20.03513640768817,1.3291054377940912
2,stddev,10.931046354331974,6.067558307506993,51.58035083014848,35.268584896496925,16.322789870993667,22.860785410540753,12.857751456762884,14.571873168680431,7.96382748276285,...,32.099634988385645,175.61888937668877,122.22879912808352,95.05063055805984,16.161764077993315,114.42790475450818,173.97733604430147,13.346556689429212,185.55824667696422,22.08857637885433
3,min,1922,1.749,-337.0925,-301.00506,-154.18358,-181.95337,-81.79429,-188.214,-72.50385,...,-437.72203,-4402.37644,-1810.68919,-3098.35031,-341.78912,-3168.92457,-4319.99232,-236.03926,-7458.37815,-381.42443
4,max,2011,61.97014,384.06573,322.85143,335.77182,262.06887,166.23689,172.40268,126.74127,...,840.97338,4469.45487,3210.7017,1734.07969,260.5449,3662.06565,2833.60895,463.4195,7393.39844,677.89963


La BD doit être un DF avec 2 colonnes
- 1 label contenant la valeur à prédire
- 2 features contenant les variables explicatives

La fonction DenseVector permet de regrouper plusieurs variables en une seule. 
Par contre il faut transformer la df en rdd.

rdd_ml = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

puis la remettre en formt df
df_ml = spark.createDataFrame(rdd_ml, ['label', 'features'])


In [7]:
from pyspark.ml.linalg import DenseVector

# créer un rdd à partir de la df en séparant la colonne à prédire des autres variables 
rdd_ml = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# mettre le rdd sous formt df en nommant les deux colonnes
df_ml = spark.createDataFrame(rdd_ml, ['label', 'features'])

df_ml.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 2001|[49.94357,21.4711...|
| 2001|[48.73215,18.4293...|
| 2001|[50.95714,31.8560...|
| 2001|[48.2475,-1.89837...|
| 2001|[50.9702,42.20998...|
| 2001|[50.54767,0.31568...|
| 2001|[50.57546,33.1784...|
| 2001|[48.26892,8.97526...|
| 2001|[49.75468,33.9958...|
| 2007|[45.17809,46.3423...|
| 2008|[39.13076,-23.017...|
| 2002|[37.66498,-34.059...|
| 2004|[26.51957,-148.15...|
| 2003|[37.68491,-26.841...|
| 1999|[39.11695,-8.2976...|
| 2003|[35.05129,-67.977...|
| 2002|[33.63129,-96.149...|
| 1992|[41.38639,-20.786...|
| 1997|[37.45034,11.4261...|
| 1987|[39.71092,-4.928,...|
+-----+--------------------+
only showing top 20 rows



## Séparer la BD en deux: train et test

In [8]:
(train, test) = df_ml.randomSplit([.8, .2], seed = 222)

train.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 1922|[39.96727,41.8845...|
| 1922|[40.96435,64.5129...|
| 1922|[41.02674,31.1615...|
| 1926|[27.59278,-179.29...|
| 1926|[31.08514,-146.64...|
| 1926|[31.66611,-174.42...|
| 1927|[26.05789,-213.38...|
| 1927|[32.80382,-165.04...|
| 1927|[34.45029,-124.70...|
| 1927|[35.6517,-137.696...|
| 1927|[36.72843,-73.046...|
| 1927|[38.55108,-68.016...|
| 1928|[32.33556,-200.12...|
| 1928|[34.5108,-241.287...|
| 1928|[34.74756,-60.680...|
| 1928|[35.49673,-230.03...|
| 1928|[36.89361,-74.548...|
| 1928|[37.09369,-119.24...|
| 1929|[22.43376,-106.81...|
| 1929|[28.86846,-134.80...|
+-----+--------------------+
only showing top 20 rows



In [9]:
print("Training data set count: " + str(train.count()))
print("Test data set count: " + str(test.count()))

Training data set count: 412014
Test data set count: 103331


In [10]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

clf = RandomForestClassifier(labelCol='label', featuresCol='features')

In [ ]:
rfModel = clf.fit(train)

Les données sont prêtes

## Appliquer la fonction régression linéaire sur cette base de données

In [9]:
from pyspark.ml.regression import LinearRegression

# créer la fonction avec les paramètres adaptés
lr = LinearRegression(labelCol='label', featuresCol='features')

# appliquer la fonction aux données d'entraînement
linearModel = lr.fit(train)

## Evaluer le modèle

### La méthode transform permet d'effectuer des prédictions en utilisant le modèle précédemment entraîné sur les données test

In [10]:
predicted = linearModel.transform(test)

predicted.show()

+-----+--------------------+------------------+
|label|            features|        prediction|
+-----+--------------------+------------------+
| 1922|[43.68703,39.4915...|1992.6693394653053|
| 1922|[46.15136,66.0833...|1997.8178673921616|
| 1926|[30.93702,-134.48...|1985.6612526954189|
| 1926|[33.56654,-108.02...| 1984.351345826122|
| 1927|[32.05226,-212.58...|1999.4042605274938|
| 1928|[23.30325,-161.61...| 1992.270116806092|
| 1928|[31.04544,-226.47...|1995.3462977548033|
| 1929|[26.64447,-72.858...|1982.1302096572354|
| 1929|[35.07089,-67.509...|1992.8264528847812|
| 1929|[35.52744,-147.45...|1989.2155707594618|
| 1929|[36.34358,-137.32...|1985.2056367144933|
| 1929|[37.26627,-54.121...|1986.9368287206887|
| 1929|[37.30241,-69.688...|1986.1068673930301|
| 1929|[38.38185,-55.354...|1994.1931199143758|
| 1930|[34.14566,-89.762...|1985.2599625914706|
| 1930|[38.10036,-65.865...|1988.6101254063515|
| 1930|[39.41802,-73.499...|1984.9167263500447|
| 1930|[41.08332,-93.379...|  1989.19393

In [11]:
print(f"RMSE: {linearModel.summary.rootMeanSquaredError}")
print(f"R2: {linearModel.summary.r2}")

RMSE: 9.549170611597843
R2: 0.2370793808427959


Le R2 est faible. Donc le modèle de régression linéaire n'est pas adapté. Il faut utiliser un autre modèle.

Ne pas oublier de femer la session spark.

In [12]:
spark.stop()